In [ ]:
import os
import glob
import gzip
import glob

In [ ]:
def decompress_gzip(input_filepath, output_filepath):
    input_files = glob.glob()

    for f_path in input_files:
        with gzip.open(f_path, 'rb') as f_in:
            with open(output_filepath, 'wb') as f_out:
                f_out.write(f_in.read())

def decompress_gz_files(files_gz_path, path_to_extract_files):
    gz_files = glob.glob(os.path.join(files_gz_path, '*.faa.gz'))

    for gz_file in gz_files:
        os.system(f'gunzip -c {gz_file} > {os.path.join(path_to_extract_files, os.path.basename(gz_file)[:-7] + ".fna")}')